In [137]:
import random
import datasets
from transformers import BertTokenizer, BertForMaskedLM
import torch
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
import codecs
import os

In [ ]:
def process_prompt(prompt):
    return prompt.replace('[ WP ]', '').replace('[ OT ]', '').replace('[ IP ]', '').replace('[ HP ]', '').replace('[ TT ]', '').replace('[ Punch ]', '').replace('[ FF ]', '').replace('[ CW ]', '').replace('[ EU ]', '')

def process_spaces(story):
    return story.replace(
        ' ,', ',').replace(
        ' .', '.').replace(
        ' ?', '?').replace(
        ' !', '!').replace(
        ' ;', ';').replace(
        ' \'', '\'').replace(
        ' ’ ', '\'').replace(
        ' :', ':').replace(
        '<newline>', '\n').replace(
        '`` ', '"').replace(
        ' \'\'', '"').replace(
        '\'\'', '"').replace(
        '.. ', '... ').replace(
        ' )', ')').replace(
        '( ', '(').replace(
        ' n\'t', 'n\'t').replace(
        ' i ', ' I ').replace(
        ' i\'', ' I\'').replace(
        '\\\'', '\'').replace(
        '\n ', '\n').strip()


writing_path = 'data/writingPrompts'

with open(f'{writing_path}/valid.wp_source', 'r', encoding='utf-8') as f:
    prompts = f.readlines()
with open(f'{writing_path}/valid.wp_target', 'r', encoding='utf-8') as f:
    stories = f.readlines()

prompts = [process_prompt(prompt) for prompt in prompts]
stories = [process_spaces(story) for story in stories]
joined = ["Prompt: " + prompt + " Story: " + story for prompt, story in zip(prompts, stories)]
filtered = [story for story in joined if 'nsfw' not in story and 'NSFW' not in story]

long_data = [x for x in filtered if len(tokenizer(x)['input_ids']) <= 512]
if len(long_data) > 0:
    filtered = long_data

# Create a list of tuples where each tuple is a pair of a prompt and a corresponding story
data = list(dict.fromkeys(data))  # Remove duplicate entries

print(f"Loaded {len(data)} prompts and stories from the writingPrompts dataset")


In [ ]:
print(type(data))